<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-spring-2018) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score
from scipy import sparse

In [2]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [5]:
train['Route'] = train['Origin'] + train['Dest']


In [6]:
test['Route'] = test['Origin'] + test['Dest']

In [7]:
X_train = train.drop('dep_delayed_15min', axis=1)
y_train = train['dep_delayed_15min'].map({'N': 0, 'Y': 1})
X_test = test.copy()

In [12]:
category_features = [id for id, col in enumerate(X_train.dtypes) if col == np.dtype('O')]
digit_features = [id for id, col in enumerate(X_train.dtypes) if col == np.dtype('int64')]

In [9]:
for col in X_train.columns[category_features]:
    labelEncoder = LabelEncoder().fit(X_train[col].append(X_test[col]))
    X_train[col] = labelEncoder.transform(X_train[col])
    X_test[col] = labelEncoder.transform(X_test[col])

ohe = OneHotEncoder(handle_unknown='ignore', sparse=True).fit(X_train.iloc[:, category_features])
X_train_ohe = ohe.transform(X_train.iloc[:, category_features])
X_test_ohe = ohe.transform(X_test.iloc[:, category_features])

X_train_matrix = []
X_test_matrix = []
for col in digit_features:
    ss = StandardScaler().fit(X_train.iloc[:, col].values.reshape(-1, 1))
    X_train_matrix.append(sparse.csr_matrix(ss.transform(X_train.iloc[:, col].values.reshape(-1, 1))))
    X_test_matrix.append(sparse.csr_matrix(ss.transform(X_test.iloc[:, col].values.reshape(-1, 1))))
    
X_train_total = sparse.hstack([X_train_ohe, sparse.hstack(X_train_matrix)])
X_test_total = sparse.hstack([X_test_ohe, sparse.hstack(X_test_matrix)])

In [10]:
cl = XGBClassifier(n_estimators=500, learning_rate=0.1, random_state=17).fit(X_train_total, y_train)
y_pred = cl.predict_proba(X_test_total)[:, 1]
roc_auc_score(y_train, y_pred)
pd.Series(y_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)